# python modules

In [1]:
from os import rename, times
# from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import re
import numpy as np
import json
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


# loding data

In [2]:
# creating model
df = pd.read_csv('merged-1-to-16.csv')
#remove data of unrated round like technocup etc.....
# df=df.reset_index()
# print(df.head())
# droping all row having any negative value in column
df = df[df.select_dtypes(include=[np.number]).ge(0).all(1)]
# droping row having avg solving time is gratar than 10 hr and less than equal to 0
df = df.loc[(df['avg_solving_time'] <= 18000) & (df['avg_solving_time'] > 0)]
df_div1 = df.loc[(df['contest_type'] == 1)]
df_div2 = df.loc[(df['contest_type'] == 2)]
df_div3 = df.loc[(df['contest_type'] == 3)]
# df_div3.describe()
# df.head()
# df.describe()
# print(df.head())
print(df.describe())
q_1 = df.loc[:, ['question_no', 'q_rating', 'contest_type']]
q_1 = q_1.loc[(q_1['question_no'] == 1) & (q_1['contest_type'] == 1)]
# print(q_1)
# print(q_1.describe())
x = df.drop(['q_rating'], axis=1)
x = x.drop(['contest_id'], axis=1)
y = df['q_rating']


        contest_id     q_rating  contest_type  question_no     accuracy  \
count  4700.000000  4700.000000   4700.000000  4700.000000  4700.000000   
mean   1102.369149  1891.595745      2.234894     3.822340     0.634800   
std     251.454666   677.331562      1.077807     2.253873     0.258493   
min     641.000000   800.000000      0.000000     1.000000     0.002401   
25%     886.000000  1400.000000      2.000000     2.000000     0.457588   
50%    1103.500000  1900.000000      2.000000     4.000000     0.678199   
75%    1322.000000  2400.000000      3.000000     5.000000     0.853926   
max    1529.000000  3500.000000      4.000000    16.000000     1.000000   

       avg_solving_time    sub_ratio  tried_but_cannot_solved  \
count       4700.000000  4700.000000              4700.000000   
mean        2451.258099     0.327319               446.673830   
std         1302.181881     0.330178               694.320172   
min            9.000000     0.000070                 0.000000   

# custum code for extracting data of sigle constes

In [24]:
data = pd.DataFrame(columns=['contest_id', 'q_rating', 'contest_type', 'question_no', 'accuracy',
                             'avg_solving_time', 'sub_ratio', 'tried_but_cannot_solved', 'total_participant',
                             'total_correct_sub_for_each_q', 'total_sub_for_each_q', 'total_time_to_sove_q'])
#sub_ratio=correct sub/total participnat
#accuracy=correct sub/total sub
#avg sol time=sum of time of accepted sol/no of accepted sol
id = "1592"
# print(cnt,id)
# id="1526"
standing = requests.get(
    "https://codeforces.com/api/contest.standings?contestId="+id)

# print(standing.json()
json_file = standing.json()
re = json_file['result']['rows']
contest_name = json_file['result']['contest']['name']
# useful data
total_participant = len(re)
total_correct_sub_for_each_question = {}
problem_rating = {}
tried_but_not_solved = {}
total_time_to_solve_q = {}
total_sub_for_each_question = {}
avg_solve_time_for_each_question = {}
total_no_of_question = 0
accuracy = {}

q_list = json_file['result']['problems']
for i in q_list:
  # print(i['index'],i['rating'])
  if 'rating' in i:
    problem_rating[total_no_of_question+1] = i['rating']
  else:
    problem_rating[total_no_of_question+1] = -1
  total_no_of_question += 1
  total_time_to_solve_q[total_no_of_question] = 0
  total_correct_sub_for_each_question[total_no_of_question] = 0
  tried_but_not_solved[total_no_of_question] = 0
  total_sub_for_each_question[total_no_of_question] = 0
  accuracy[total_no_of_question] = 0
# print(problem_rating)
# print(total_no_of_question)
# print(re)
for i in re:
  user_sub_for_each_q = i['problemResults']
  question_id = 1
  temp = []
  for i in user_sub_for_each_q:
    if('bestSubmissionTimeSeconds' in i):
      temp.append([i['bestSubmissionTimeSeconds'], question_id])
      total_correct_sub_for_each_question[question_id] += 1
    else:
      temp.append([0, question_id])
      if i['rejectedAttemptCount'] > 0:
        tried_but_not_solved[question_id] += 1
    question_id += 1
  temp.sort()
  prev_time = 0
  for [x, y] in temp:
    total_time_to_solve_q[y] += x-prev_time
    prev_time = x
  # print(total_time_to_solve_q)
for i in range(total_no_of_question):
  total_sub_for_each_question[i +
                              1] = total_correct_sub_for_each_question[i+1]+tried_but_not_solved[i+1]
  if total_sub_for_each_question[i+1] > 0:
    accuracy[i+1] = total_correct_sub_for_each_question[i+1] / \
        total_sub_for_each_question[i+1]

#output
# print(total_participant)
# print(total_correct_sub_for_each_question)
# print(total_sub_for_each_question)
# print(tried_but_not_solved)
# print(total_time_to_solve_q)
# print("avg solving time in second")
for i in range(total_no_of_question):
  if(total_correct_sub_for_each_question[i+1] > 0):
    avg_solve_time_for_each_question[i+1] = total_time_to_solve_q[i+1] / \
        total_correct_sub_for_each_question[i+1]
  else:
    avg_solve_time_for_each_question[i+1] = 0
# print(avg_solve_time_for_each_question)
contest_type = 4  # for all other contest
if "Div. 3" in contest_name:
  contest_type = 3
if "Div. 2" in contest_name:
  contest_type = 2
if "Div. 1" in contest_name:
  contest_type = 1
if "Global Round" in contest_name:
  contest_type = 0
for i in range(total_no_of_question):
  data = data.append({'contest_id': id, 'q_rating': problem_rating[i+1], 'contest_type': contest_type, 'question_no': i+1, 'accuracy': accuracy[i+1],
                      'avg_solving_time': avg_solve_time_for_each_question[i+1], 'sub_ratio': total_correct_sub_for_each_question[i+1]/total_participant,
                      'tried_but_cannot_solved': tried_but_not_solved[i+1], 'total_participant': total_participant, 'total_correct_sub_for_each_q':
                      total_correct_sub_for_each_question[i+1], 'total_sub_for_each_q': total_sub_for_each_question[i+1], 'total_time_to_sove_q':
                      total_time_to_solve_q[i+1]}, ignore_index=True)
# print(data)
data = data.drop(['q_rating'], axis=1)
print(data)
# print(taken)


  contest_id contest_type question_no  accuracy  avg_solving_time  sub_ratio  \
0       1592            2           1  0.801662       1471.819655   0.785313   
1       1592            2           2  0.669834       2034.840284   0.435173   
2       1592            2           3  0.559815       2902.853960   0.113377   
3       1592            2           4  0.339250       3085.116279   0.016090   
4       1592            2           5  0.280702       3152.575000   0.007484   
5       1592            2           6  0.500000       1307.933333   0.001403   
6       1592            2           7  0.666667       1621.500000   0.000561   

  tried_but_cannot_solved total_participant total_correct_sub_for_each_q  \
0                    2077             10690                         8395   
1                    2293             10690                         4652   
2                     953             10690                         1212   
3                     335             10690            

# traning model (random forest alog)

In [3]:
model = RandomForestRegressor()
train_x_forest, test_x_forest, train_y_forest, test_y_forest = train_test_split(
    x, y, test_size=0.2, random_state=42)
model.fit(train_x_forest, train_y_forest)
y_predict_forest = model.predict(test_x_forest)


# output for div 1

In [12]:
x_div1=df_div1.drop(['q_rating'],axis=1)
# x_div1=x_div1.drop(['contest_id'],axis=1)
y_div1=df_div1['q_rating']
# print(x_div1.head())
# print(y_div1.head())
model=RandomForestRegressor()
train_x_forest_div1,test_x_forest_div1,train_y_forest_div1,test_y_forest_div1=train_test_split(x_div1,y_div1,test_size=0.2,random_state=42)
model.fit(train_x_forest_div1,train_y_forest_div1)
print(test_x_forest_div1.head())
y_predict_forest_div1=model.predict(test_x_forest_div1)
# test_x_forest_div11=[[1,5,0.056	,4440,0.00709219858,22,1,23]]
# y_predict_forest_div11=model.predict(test_x_forest_div11)
# print(y_predict_forest_div11)
# y_predict


      contest_id  contest_type  question_no  accuracy  avg_solving_time  \
4771         715             1            1  0.982249       1704.168675   
3058        1037             1            6  0.565543       2592.940397   
3147        1010             1            1  0.753022        942.013761   
3624         933             1            4  0.761905       4361.812500   
3535         949             1            2  0.969828       2295.920000   

      sub_ratio  tried_but_cannot_solved  total_participant  \
4771   0.930841                        9                535   
3058   0.030164                      116               5006   
3147   0.742760                      143                587   
3624   0.029575                        5                541   
3535   0.875486                       14                514   

      total_correct_sub_for_each_q  total_sub_for_each_q  total_time_to_sove_q  
4771                           498                   507                848676  
3058    

# output for div 2

In [25]:
x_div2 = df_div2.drop(['q_rating'], axis=1)
# x_div2=x_div2.drop(['contest_id'],axis=1)
y_div2 = df_div2['q_rating']
model = RandomForestRegressor()
train_x_forest_div2, test_x_forest_div2, train_y_forest_div2, test_y_forest_div2 = train_test_split(
    x_div2, y_div2, test_size=0.05, random_state=42)
# print(train_x_forest_div2.describe())
model.fit(train_x_forest_div2, train_y_forest_div2)
y_predict_forest_div2 = model.predict(test_x_forest_div2)
# y_predict
y_predict_forest_div2 = model.predict(data)
y_predict_forest_div2


array([ 815., 1256., 1800., 2093., 2270., 2582., 2773.])

# output div 3

In [19]:
x_div3 = df_div3.drop(['q_rating'], axis=1)
# x_div2=x_div2.drop(['contest_id'],axis=1)
y_div3 = df_div3['q_rating']
model = RandomForestRegressor()
train_x_forest_div3, test_x_forest_div3, train_y_forest_div3, test_y_forest_div3 = train_test_split(
    x_div3, y_div3, test_size=0.2, random_state=42)
model.fit(train_x_forest_div3, train_y_forest_div3)
y_predict_forest_div3 = model.predict(test_x_forest_div3)
# y_predict
